In [1]:
import pandas as pd

In [6]:
reviews = pd.read_table("amazon_data/sampled_reviews_small.tsv",error_bad_lines=False,low_memory=False)

b'Skipping line 20114: expected 15 fields, saw 22\nSkipping line 36320: expected 15 fields, saw 22\nSkipping line 38067: expected 15 fields, saw 22\nSkipping line 40844: expected 15 fields, saw 22\nSkipping line 111102: expected 15 fields, saw 22\nSkipping line 128910: expected 15 fields, saw 22\n'


In [7]:
reviews.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

In [4]:
num_prod = len(reviews['product_category'].unique())

KeyError: 'product_category'

In [73]:
num_prod

53

In [74]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import string
from nltk.corpus import stopwords
import re
from scipy.sparse import csr_matrix
from gensim.models import Word2Vec
import logging

In [96]:
_BOS = "_BOS "
_EOS = " _EOS"
DIM_SIZE = 300
w2v_model = Word2Vec.load("amazon_data/model.en")

In [84]:
def remove_punct(sentence):
    #sentence = re.sub(stop, " ", sentence) 
    return sentence.lower().translate(str.maketrans('', '', string.punctuation))

def create_dictionary(data_path):
    reviews = pd.read_table(data_path,error_bad_lines=False,low_memory=False)

    input_texts = [] 
    target_texts = []
    prod_category = []
    # idx2word
    input_title_vocab = set()
    target_review_vocab = set()
    #
    for index,row in reviews.iterrows():
        attributes = [row["product_title"],row["review_headline"],row["review_body"]]
        if not isinstance(row["product_title"],str): 
            continue
        if not isinstance(row["review_headline"],str): 
            continue
        if not isinstance(row["review_body"],str): 
            continue   
        input_text, target_text = remove_punct(row["product_title"]),\
        remove_punct(" ".join([row["review_headline"],row["review_body"]]))
        target_text = _BOS + target_text + _EOS
        input_texts.append(input_text)
        target_texts.append(target_text)
        prod_category.append(row["product_category"])
        for word in input_text.split():
            if word not in input_title_vocab:
                input_title_vocab.add(word)
        for word in target_text.split():
            if word not in target_review_vocab:
                target_review_vocab.add(word)

    return input_texts, target_texts, input_title_vocab, target_review_vocab, prod_category

input_texts, target_texts, input_title_vocab, target_review_vocab, prod_category = create_dictionary("amazon_data/sampled_reviews.tsv")

b'Skipping line 128328: expected 15 fields, saw 22\nSkipping line 134749: expected 15 fields, saw 22\nSkipping line 174759: expected 15 fields, saw 22\nSkipping line 197766: expected 15 fields, saw 22\nSkipping line 202569: expected 15 fields, saw 22\nSkipping line 382089: expected 15 fields, saw 22\nSkipping line 397816: expected 15 fields, saw 22\nSkipping line 398935: expected 15 fields, saw 22\nSkipping line 416875: expected 15 fields, saw 22\nSkipping line 431901: expected 15 fields, saw 22\nSkipping line 554909: expected 15 fields, saw 22\nSkipping line 657304: expected 15 fields, saw 22\nSkipping line 691801: expected 15 fields, saw 22\nSkipping line 711377: expected 15 fields, saw 22\nSkipping line 741194: expected 15 fields, saw 22\nSkipping line 772352: expected 15 fields, saw 22\nSkipping line 775569: expected 15 fields, saw 22\nSkipping line 778834: expected 15 fields, saw 22\nSkipping line 779740: expected 15 fields, saw 22\nSkipping line 789067: expected 15 fields, saw 22

In [85]:
vocab = sorted(list(input_title_vocab.union(target_review_vocab)))
target_review_vocab = sorted(list(target_review_vocab))
# the number of sample vocablary
encoder_vocab_size = len(input_title_vocab)
decoder_vocab_size = len(target_review_vocab)
vocab_size = len(vocab)
# Define max length of encoder / decoder
#max_encoder_seq_length = max([len(text.split()) for text in input_texts])
#max_decoder_seq_length = max([len(text.split()) for text in target_texts])
input_di_text,target_di_text = {},{}
text_index = 0
max_seq_length = 0
for text in input_texts:
    if len(text.split()) <= 100:
        input_di_text[text_index] = text
        text_index += 1
        max_seq_length = max(max_seq_length,len(text.split()))
for text in target_texts:
    if len(text.split()) <= 100:
        target_di_text[text_index] = text
        text_index += 1
        max_seq_length = max(max_seq_length,len(text.split()))
    

num_prod = len(reviews['product_category'].unique())
inverse_input_vocab = dict(
    [(word, id) for id, word in enumerate(input_title_vocab)])
inverse_target_vocab = dict(
    [(word, id) for id, word in enumerate(target_review_vocab)])
inverse_vocab = dict(
    [(word, id) for id, word in enumerate(vocab)])
inverse_prod_category = dict(
    [(word, id) for id, word in enumerate(reviews['product_category'].unique())])

In [86]:
from keras.callbacks import TensorBoard

In [63]:
encoder_input_data = np.zeros(
    (len(input_di_text), max_seq_length, DIM_SIZE),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(target_di_text), max_seq_length, DIM_SIZE),
   dtype='float32')
decoder_target_data = np.zeros(
    (len(target_di_text), max_seq_length, DIM_SIZE),
    dtype='float32')
prod_category_data = np.zeros(
    (len(prod_category), 1, num_prod),
    dtype='float32')

for pair_text_idx, (input_text, target_text,prod_cat) in enumerate(zip(input_texts, target_texts,prod_category)):
    for timestep, word in enumerate(input_text.split()):
        encoder_input_data[pair_text_idx, timestep, inverse_vocab[word]] = 1.
    # decoder_target_data is ahead of decoder_input_data by one timestep
    for timestep, word in enumerate(target_text.split()):
        decoder_input_data[pair_text_idx, timestep, inverse_vocab[word]] = 1.
        if timestep > 0:
            # decoder_target_data will be ahead by one timestep（LSTM)
            # decoder_target_data will not include the start character.
            decoder_target_data[pair_text_idx, timestep - 1, inverse_vocab[word]] = 1.
    prod_category_data[pair_text_idx,0, inverse_prod_category[prod_cat]] = 1.

MemoryError: Unable to allocate array with shape (1520031, 100, 600) and data type float32

In [95]:
NUM_HIDDEN_UNITS = 128 # NUM_HIDDEN_LAYERS
BATCH_SIZE = 512
NUM_EPOCHS = 40

In [88]:
import tensorflow as tf
class Attention(tf.keras.Model):

    def __init__(self, units):
        # initialize initial values
        super(Attention, self).__init__()
        # architecture of nn
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hudden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        prob_score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        attention_weights = tf.nn.softmax(self.V(prob_score), axis=1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

In [97]:
input_texts = list(input_di_text.values())
target_texts = list(target_di_text.values())
def batch_generator(batch_size):
    while True:
        encoder_input_data = np.zeros(
            (batch_size, max_seq_length, DIM_SIZE),
            dtype='float32')
        decoder_input_data = np.zeros(
            (batch_size, max_seq_length, DIM_SIZE),
            dtype='float32')
        decoder_target_data = np.zeros(
            (batch_size, max_seq_length, DIM_SIZE),
            dtype='float32')
        prod_category_data = np.zeros(
            (batch_size, 1, num_prod),
            dtype='float32')
        n_batches_for_epoch = len(input_di_text)//batch_size
        for i in range(n_batches_for_epoch):
            for pair_text_idx, (input_text, target_text,prod_cat) in enumerate(zip(input_texts[batch_size*i:batch_size*(i+1)], target_texts[batch_size*i:batch_size*(i+1)],prod_category[batch_size*i:batch_size*(i+1)])):
                for timestep, word in enumerate(input_text.split()):
                    if word in w2v_model.wv:
                        encoder_input_data[pair_text_idx, timestep] = w2v_model.wv[word]
                # decoder_target_data is ahead of decoder_input_data by one timestep
                for timestep, word in enumerate(target_text.split()):
                    if word in w2v_model.wv:
                        decoder_input_data[pair_text_idx, timestep] = w2v_model.wv[word]
                    if timestep > 0:
                        # decoder_target_data will be ahead by one timestep（LSTM)
                        decoder_target_data[pair_text_idx, timestep - 1] = w2v_model.wv[word]
                prod_category_data[pair_text_idx,0, inverse_prod_category[prod_cat]] = 1.
            yield([encoder_input_data, decoder_input_data],[decoder_target_data,prod_category_data])
#         for i in range(n_batches_for_epoch):
#             index_batch = range(X.shape[0])[batch_size*i:batch_size*(i+1)]
#             X_batch = X[index_batch,:].todense()
#             yield(X_batch, X_batch)

In [ ]:
"""
Encoder Architecture
"""

encoder_inputs = Input(shape=(None, DIM_SIZE))
encoder_lstm = LSTM(units=NUM_HIDDEN_UNITS, return_state=True)
# x-axis: time-step lstm
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c] # We discard `encoder_outputs` and only keep the states.

"""
Decoder Architecture
"""
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_inputs = Input(shape=(None, DIM_SIZE))
decoder_lstm = LSTM(units=NUM_HIDDEN_UNITS, return_sequences=True, return_state=True)
# x-axis: time-step lstm
decoder_outputs, de_state_h, de_state_c = decoder_lstm(decoder_inputs, initial_state=encoder_states) # Set up the decoder, using `encoder_states` as initial state.
decoder_softmax_layer = Dense(DIM_SIZE, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

classification_dense = Dense(100, activation = 'relu')(decoder_outputs)
classification_outputs = Dense(num_prod,activation = 'relu')(classification_dense)
"""
Encoder-Decoder Architecture
"""
# Define the model that will turn, `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], [decoder_outputs,classification_outputs])
print(model.summary())
model.compile(optimizer="rmsprop", loss=["categorical_crossentropy","binary_crossentropy"]) # Set up model
model.fit_generator(generator=batch_generator(BATCH_SIZE),
                    nb_epoch=10,
                    shuffle=True,
                    samples_per_epoch=len(input_di_text),
                    callbacks=[TensorBoard(log_dir='/tmp/autoencoder')],
                    verbose=True)

# model.fit(x=[encoder_input_data, decoder_input_data], y=[decoder_target_data,prod_category_data],
#           batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_split=0.2) # Run training
model.save("seq2seq_translate_model.txt") # Save model

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_36 (InputLayer)           (None, None, 300)    0                                            
__________________________________________________________________________________________________
input_37 (InputLayer)           (None, None, 300)    0                                            
__________________________________________________________________________________________________
lstm_35 (LSTM)                  [(None, 128), (None, 219648      input_36[0][0]                   
__________________________________________________________________________________________________
lstm_36 (LSTM)                  [(None, None, 128),  219648      input_37[0][0]                   
                                                                 lstm_35[0][1]             

/home/reddy/akirato/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/reddy/akirato/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., shuffle=True, callbacks=[<keras.ca..., verbose=True, steps_per_epoch=1520031, epochs=10)`


Epoch 1/10
    238/1520031 [..............................] - ETA: 151:15:19 - loss: -1024.2956 - dense_52_loss: -1024.5825 - dense_54_loss: 0.2870

In [ ]:
# inputs=encoder_inputs, outputs=encoder_states
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

# State from encoder
decoder_state_input_h = Input(shape=(NUM_HIDDEN_UNITS,))
decoder_state_input_c = Input(shape=(NUM_HIDDEN_UNITS,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
# x-axis: time-step lstm
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_state_inputs, outputs=[decoder_outputs] + decoder_states)



input_idx2word = dict(
    (id, word) for word, id in inverse_vocab.items())
target_idx2word = dict(
    (id, word) for word, id in inverse_vocab.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, vocab_size))
    # Populate the first character of target sequence with the start character.
    #target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_idx2word[sampled_token_index]
        decoded_sentence += " "+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_seq_length):
            stop_condition = True
        num_decoder_tokens = vocab_size
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

def classify_sequence(input_seq,decode_seq):
    print(input_seq.shape,decode_seq.shape)
    return model.predict([input_seq,decode_seq])

for seq_index in range(5):
    # Take one sequence (part of the training set) for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decode_seq = decoder_input_data[seq_index:seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    output = classify_sequence(input_seq, decode_seq)
    print("classifier:", np.argmax(output[1][0, -1, :]),[x.shape for x in output])

In [ ]:
inverse_target_vocab.keys()